### Nettoyage des questions et des réponses :

In [197]:
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import csv
import pickle
from pandas import DataFrame
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer

In [198]:
data = pd.read_csv('dataQR.csv', sep=';')

In [199]:
data.iloc[1][1]

"Acheter sur Vinted : toutes les étapes\nTrouve un article qui te plaît\nL'icône en forme de cœur te permet de sauvegarder les articles que tu aimes dans tes favoris.\nSuis tes marques et tes membres préférés, ainsi leurs articles apparaîtront dans ton fil d'actualité. Pour accéder au fil d'actualité ou à l'onglet tendance sur appli, clique simplement sur l'icône en forme de boussole situé sur l'extrémité gauche du menu de navigation.\nTu peux filtrer les articles par taille.\nUtilise les autres filtres de notre catalogue pour trier les articles par marque/couleur/prix, etc.\nCliquer sur\nTrier\n pour organiser les résultats de ta recherche par nouveauté, prix croissant, prix décroissant et cohérence avec l'article recherché.\nContacte le vendeur\nClique sur\nEnvoyer un message.\nDans la fenêtre qui s'ouvre, tu peux :\nDiscuter avec le membre.\nProposer un autre prix via le bouton\nFaire une offre.\nProcéder à l'achat immédiat de l'article.\nDemander plus d'informations au sujet de l'a

On remplace les retours à la ligne par des valeurs vides : 

In [200]:
nltk_stopwords = nltk.corpus.stopwords.words('french')
stopwords = list(nltk_stopwords)
stemmer=nltk.stem.SnowballStemmer('french')

def nettoyage_texte(txt):
    #on met tout le texte en minuscule, et on remplce les retours à la ligne par des espaces
    txt = txt.lower().replace('\n', ' ')

    #on retire tous les accents 
    txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore')

    #on garde uniquement les caractères alphanumériques : 
    txt = re.sub('[^a-z_]', ' ', str(txt))
    
    #on retire les stops words
    tokens = [w for w in txt.split() if (len(w)>2) and (w not in nltk_stopwords)]
    
    #pour avoir les mots racines
    tokens = [stemmer.stem(token) for token in tokens]

    return ' '.join(tokens)

In [201]:
with open("nettoyage_texte.pk", "wb") as f:
    b = pickle.Pickler(f)
    b.dump(nettoyage_texte)

In [202]:
data['question'] = data['question'].apply(lambda x : nettoyage_texte(x))

data['reponse'] = data['reponse'].apply(lambda x : nettoyage_texte(x))


In [203]:
data.iloc[1][1]

'achet vinted tout etap trouv articl plait icon form cur permet sauvegard articl aim favor marqu membr prefer ains leur articl apparaitront fil actualit acced fil actualit onglet tendanc appli cliqu simpl icon form boussol situ extremit gauch menu navig peux filtr articl taill utilis autr filtr catalogu tri articl marqu couleur prix etc cliqu tri organis resultat recherch nouveaut prix croiss prix decroiss coherent articl recherch contact vendeur cliqu envoi messag fenetr ouvr peux discut membr propos autr prix vi bouton fair offre proced achat immediat articl demand plus inform sujet articl achet articl cliqu achet proced pai articl lor premi achat devr entrer donne cart bancair peux regl achat cart bancair vi ideal compt paypal cas dun envoi suiv uniqu prix pai comprend frais livraison frais protect acheteur sappliquent tous pai effectu vinted verif adress livraison frais port varient dependent poid colis mod envoi chois peux ajout modifi mod pai depuis pag parametr web parametr appl

Création du Dataframe normalisé : 

In [209]:
dataQR_normalize = DataFrame({'Question_normalize' : data['question'], 'Reponse_normalize' : data['reponse']})

dataQR_normalize.to_csv('dataQR_normalize.csv', index = False, sep=';')

On construit un vectoriseur sur les questions :

In [205]:
vectorizer_q = TfidfVectorizer()

dtm_question = vectorizer_q.fit(data['question'])

with open('dtm_question.pk', 'wb') as fin:
    pickle.dump(vectorizer_q, fin)

In [206]:
#Affichage de la matrice : 
pd.DataFrame(dtm_question.transform(data['question']).toarray(), columns=vectorizer_q.get_feature_names())

,abon,accepte,achat,achet,acheteur,activ,adress,aid,ajout,amis,...,vendr,vent,verif,verifi,vers,vinted,visibl,visit,vit,vitrin
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.225255,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.303041,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.250566,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.289188,0.0,0.0,0.0,0.0,0.289188,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.204268,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.523836,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
217,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
218,0.0,0.0,0.0,0.000000,0.0,0.0,0.417919,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
219,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


On construit un vectoriseur sur les réponses : 

In [207]:
vectorizer_r = TfidfVectorizer()

dtm_reponse = vectorizer_r.fit(data['reponse'])

with open('dtm_reponse.pk', 'wb') as fin:
    pickle.dump(vectorizer_r, fin)

In [208]:
#Affichage de la matrice : 
pd.DataFrame(dtm_reponse.transform(data['reponse']).toarray(), columns=vectorizer_r.get_feature_names())

,abim,abime,abon,abonne,abord,about,abrevi,absenc,absolu,acanc,...,web,week,western,window,www,xxxxcod,yog,your,zar,zero
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.057687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217,0.0,0.0,0.0,0.0,0.076103,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
218,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
